In [37]:
using CUDA, LinearAlgebra, CUDA.CUSPARSE, CUDA.CUBLAS, SparseArrays, BenchmarkTools

In [38]:
function proj_CPU(p₀, u, β)
    return  p₀ .- ((dot(u, p₀)- β)/dot(u, u)).*u
end

proj_CPU (generic function with 1 method)

In [39]:
n = Int32(2^20)
X = CUDA.rand(n)
Y = CUDA.rand(n)
x = Array(X)
y = Array(Y)
β = Float32(1.0)


1.0f0

In [42]:
@benchmark proj_CPU(x, y, β)

BenchmarkTools.Trial: 3014 samples with 1 evaluation.
 Range (min … max):  1.084 ms …   4.372 ms  ┊ GC (min … max):  0.00% … 37.48%
 Time  (median):     1.119 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):   1.643 ms ± 798.618 μs  ┊ GC (mean ± σ):  10.84% ± 16.15%

  █▄                      ▂▅▄▂                            ▂▃▁  
  ███▅▅▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁████▅▄▁▁▁▁▁▁▃▅▃▃▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁███ █
  180 ms       Histogram: log(frequency) by time      3.68 ms <

 Memory estimate: 4.00 MiB, allocs estimate: 2.

In [47]:
@benchmark proj_CPU(X,Y, β)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  314.234 μs … 46.256 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     395.325 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   485.002 μs ±  1.836 ms  ┊ GC (mean ± σ):  0.57% ± 2.23%

                       ▄▅▇██▅▁                  ▂▂▂▁   ▁▂▃▃▂▁  ▂
  ▃▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▄▅▁███████▇▆▅▄▄▆▄▅▄▃▁▁▁▃▁▃▆█████▆▆▇███████ █
  314 μs        Histogram: log(frequency) by time       512 μs <

 Memory estimate: 3.16 KiB, allocs estimate: 56.

In [19]:
using Cthulhu
@device_code_warntype interactive=true @cuda proj_GPU(X,Y, β, n)

PTX CompilerJob of kernel proj_GPU(CuDeviceVector{Float32, 1}, CuDeviceVector{Float32, 1}, Float32, Int32) for sm_61


│ ─ %-1  = invoke proj_GPU(::CuDeviceVector{Float32, 1},::CuDeviceVector{Float32, 1},::Float32,::Int32)::Union{}
Variables
  #self#::Core.Const(proj_GPU)
  p₀::CuDeviceVector{Float32, 1}
  u::CuDeviceVector{Float32, 1}
  β::Float32
  n::Int32

Body::Union{}
    @ In[4]:2 within `proj_GPU'
1 ─     (CUDA.CUBLAS.dot)(n, u, p₀)
└──     unreachable



In [49]:
CUDA.reclaim() #LIMPA A MEMORIA DA GPU
CUDA.memory_status()  #DIZ A QUANTIDADE DE MEMÓRIA LIVRE NA GPU

Effective GPU memory usage: 17.85% (721.188 MiB/3.945 GiB)
CUDA allocator usage: 78.148 MiB
Memory pool usage: 78.148 MiB (78.148 MiB allocated, 0 bytes cached)


In [34]:
function reflexao(p₀, u, β)
    return  2 .*proj_GPU(p₀, u, β) .- p₀
end

reflexao (generic function with 1 method)

In [ ]:
function reflexao_simultanea(xₖ, A, b, n, r)
    rₖ = CUDA.zeros(r)
      for i=1:n
         rₖ .+= reflexao(xₖ, A[i,:], b[i])
    end
    return rₖ./n
end

reflexao_simultanea (generic function with 1 method)

In [ ]:
#rₖ = Vector{Float32}(undef, 1_000) #Aloca uma memória para um vetor de tamanho 1_000 na GPU

In [ ]:
#CuArray{Int}(undef, 2) - cria um array em pé de 2 entradas

In [ ]:
#CuArray{Int}(undef, (1,2))- cria um array deitado de 2 entradas

In [ ]:
#fill!(rₖ, 0.) #Atribui a memória alocada um vetor cujas etradas são todas nulas

In [ ]:
#@sync - pausa as tarefas da CPU até as tarefas da GPU dentro do Bloco serem concluídas

In [ ]:
#@btime nome da função - mede o tempo como benchmarktools

In [ ]:
#@cuprintln("thread $index, block $stride") - imprime

In [ ]:
#synchronize() - sincroniza a GPU, necessário usar com o @cuprint()

In [ ]:
#broadcast - Faz operções com elementos que não possuem a mesma dimensão como somar um vetor 
#nas colunas de uma matriz por exemplo, com strings faz concatenação

In [ ]:
#map(f, c) -> coleção -Transformar a colecção c através da aplicação de f a cada elemento. Para múltiplos
#argumentos de recolha, aplicar f elemento a elemento. Ex: aplica uma função nas entradas de um vetor,
#opera com vetores de mesmo tamanho.

In [ ]:
#a = reshape(Vector(1:16), (4,4)) # cria o vetor de 1 até 16 depois transforma numa matriz 4x4 por colunas

#reduce(max, a, dims=2) -verifica qual o vetor de maior valores na matriz e devolve somente este vetor 
#em forma de matriz em pé

#reduce(max, a, dims=1)- verifica qual o vetor de maior valores na matriz e devolve somente este vetor 
#em forma de matriz deitada

In [ ]:
#reduce(*, [2; 3; 4]) devolve a multiplicação das entradas do vetor considerando o elemento neutro da 
#multiplicação como sendo 1

#reduce(*, [2; 3; 4]; init=-1)devolve a multiplicação das entradas do vetor considerando o elemento neutro da 
#multiplicação como sendo -1

In [ ]:
#tamanho = length(a)/1024 - em que 1024 é o numero de threads
#@cuda threads=length(a)/tamanho função(a)  - Faz a divisão para as threads

In [ ]:
#a = CuArray([1,2]) - array na GPU
#b = Array(a) - array na CPU
#copyto!(b, a) - aloca b na GPU no lugar de a

In [ ]:
#SparseArrays em cuda
A1 = sprand(10,10,0.2) #cria matriz espasa 10x10 com esparcidade 0.8 e distribuição normal

10×10 SparseMatrixCSC{Float64, Int64} with 18 stored entries:
  ⋅          ⋅         ⋅        …   ⋅         ⋅         ⋅         ⋅ 
  ⋅          ⋅         ⋅           0.27646    ⋅         ⋅         ⋅ 
 0.0930857   ⋅         ⋅            ⋅         ⋅         ⋅         ⋅ 
  ⋅          ⋅        0.254801      ⋅        0.878417   ⋅         ⋅ 
  ⋅          ⋅        0.769543      ⋅         ⋅         ⋅         ⋅ 
  ⋅          ⋅        0.567103  …  0.212777   ⋅         ⋅         ⋅ 
  ⋅          ⋅         ⋅           0.614005  0.07659    ⋅         ⋅ 
  ⋅         0.420396   ⋅            ⋅         ⋅         ⋅         ⋅ 
  ⋅          ⋅         ⋅            ⋅         ⋅         ⋅         ⋅ 
  ⋅          ⋅        0.383552      ⋅         ⋅        0.995818   ⋅ 

In [ ]:
x1 = sprand(10,0.2) #cria vetor esparço 10x1 com distribuição normal e esparsidade 0.8

10-element SparseVector{Float64, Int64} with 2 stored entries:
  [7 ]  =  0.385573
  [8 ]  =  0.243255

In [ ]:
 x(x)